In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

  Activating environment at `~/Documents/planar_quadruped_landing/Project.toml`


WebIO._IJuliaInit()

reference_trajectory

## Reference Trajectory

## Problem Definition

In [2]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 1.0
dt = 0.05
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.5

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb/2                  # xb
xinit[2] = sqrt(l1^2+l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -4.0                   # yb_dot
xinit[12] = -4.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb/2           # xb
xterm[2] = sqrt(l1^2+l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([10.0;10.0;1.0; 10.0;10.0; 10.0;10.0; 10.0;10.0;1.0; 10.0;10.0; 10.0;10.0])
R = Diagonal(fill(1e-3,4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [3]:
# Initial guess
Random.seed!(1)
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
Z0 = packZ(nlp, Xguess, Uguess);
# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

In [4]:
# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

List of options:

                                    Name   Value                # times used
            check_derivatives_for_naninf = yes                       1
                         constr_viol_tol = 0.001                     2
                   hessian_approximation = limited-memory            7
                                max_iter = 10000                     1
                      obj_scaling_factor = 1                         1
                             print_level = 8                         2
                                     tol = 0.001                     2

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
*********

LoadError: InterruptException: